In [404]:
from fractions import Fraction
import re
import sympy as sp
import numpy as np
from scipy.optimize import minimize
from typing import *

In [405]:
def optimize_equations(symbols: List[sp.Symbol], equations: List[sp.Expr],
                       initial_guess: Union[List[float],np.ndarray[float]]) -> np.ndarray:
    """
    Given a list of symbols, a list of equations, and an initial guess, optimize the values of the symbols
    that minimize the sum of squares of the equations using the scipy.optimize.minimize function.
    Returns an array of optimized values of the symbols.

    Args:
        symbols: A list of sympy symbols to optimize.
        equations: A list of sympy expressions representing the equations to solve.
        initial_guess: A list of initial guesses for the values of the symbols.

    Returns:
        An array of optimized values of the symbols.

    Raises:
        ValueError: If the length of symbols and initial_guess does not match.
    """
    if len(symbols) != len(initial_guess):
        raise ValueError("Length of symbols and initial_guess must match")

    # Create a function that takes in the values of the symbols and returns the equation
    f = sp.lambdify(symbols, equations)

    # Define the optimization problem
    _bounds = [(0, 1) for _var in symbols]
    _problem = minimize(lambda x: np.sum(np.square(f(*x))), initial_guess, bounds=_bounds)

    # Print the optimized values
    for i in range(len(symbols)):
        print(f'{symbols[i]}: {np.round(_problem.x[i], 6)}')

    return _problem.x

# 2 tickets - 3 persons:

In [406]:
p11, p21, p31 = sp.symbols("p11, p21, p31")
p22, p32      = sp.symbols("p22, p32")

vars = [p11, p21, p22, p31, p32]
# only 1 of 2 tickets bought amongst 3 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)
# 2 of 2 tickets bought amongst 3 persons
term4 = (p11+p22)*(1-p11)*(1-p22)
term5 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)
term6 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)
expectation = term1+term2+term3+term4+term5+term6

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("3persons2tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(3,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.554688
p21: 0.5
p22: 0.625
p31: 0.5
p32: 0.5

 == Corresponding to: == 
[[0.55469 0.     ]
 [0.5     0.625  ]
 [0.5     0.5    ]]


# 2 tickets - 4 persons:

In [407]:
p11, p21, p31, p41 = sp.symbols('p11, p21, p31, p41')
p22, p32, p42      = sp.symbols('p22, p32, p42')

vars = [p11, p21, p31, p41, p22, p32, p42]

# Only 1 of 3 tickets bought amongst 4 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))
term4 = p41*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)

# 2 of 2 tickets bought amongst 4 persons
term5 = (p11+p22)*(1-p11)*(1-p22)
term6 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)
term7 = (p11+p42)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-p42)
term8 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)
term9 = (p21+p42)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-p42)
term10 = (p31+p42)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-p42)

# Expectation
expectation = term1 + term2 + term3 + term4 + \
              term5 + term6 + term7 + term8 + \
              term9 + term10

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("4persons2tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)
table = np.zeros(shape=(4,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.607422
p21: 0.554686
p31: 0.5
p41: 0.5
p22: 0.695312
p32: 0.625
p42: 0.5

 == Corresponding to: == 
[[0.60742 0.     ]
 [0.55469 0.69531]
 [0.5     0.625  ]
 [0.5     0.5    ]]


# 3 Tickets - 4 persons:

In [408]:
p11, p21, p31, p41 = sp.symbols('p11, p21, p31, p41')
p22, p32, p42      = sp.symbols('p22, p32, p42')
p33, p43           = sp.symbols('p33, p43')

vars = [p11, p21, p31, p41, p22, p32, p42, p33, p43]

# 1 of 3 tickets bought amongst 4 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))
term4 = p41*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)

# 2 of 3 tickets bought amongst 4 persons
term5 = (p11+p22)*(1-p11)*(1-p22)*(1-(1-p33))*(1-(1-p43))
term6 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)*(1-(1-p43))
term7 = (p11+p42)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-p42)
term8 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)*(1-(1-p43))
term9 = (p21+p42)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-p42)
term10 = (p31+p42)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-p42)

# 3 of 3 tickets bought amongst 4 persons
term11 = (p11+p22+p33)*(1-p11)*(1-p22)*(1-p33)
term12 = (p11+p22+p43)*(1-p11)*(1-p22)*(1-(1-p33))*(1-p43)
term13 = (p11+p32+p43)*(1-p11)*(1-(1-p22))*(1-p32)*(1-p43)
term14 = (p21+p32+p43)*(1-(1-p11))*(1-p21)*(1-p32)*(1-p43)

# Expectation
expectation = term1 + term2 + term3 + term4 + \
              term5 + term6 + term7 + term8 + \
              term9 + term10 + term11 + term12 + \
              term13 + term14

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("4persons3tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)
table = np.zeros(shape=(4,3))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.525848
p21: 0.5
p31: 0.5
p41: 0.5
p22: 0.554691
p32: 0.5
p42: 0.5
p33: 0.625006
p43: 0.5

 == Corresponding to: == 
[[0.52585 0.      0.     ]
 [0.5     0.55469 0.     ]
 [0.5     0.5     0.62501]
 [0.5     0.5     0.5    ]]


# 2 tickets - 5 persons: (maybe wrong equations - double check)


In [409]:
p11, p21, p31, p41, p51 = sp.symbols('p11, p21, p31, p41, p51')
p22, p32, p42, p52      = sp.symbols('p22, p32, p42, p52')


vars = [p11, p21, p31, p41, p51, p22, p32, p42, p52]

# Only 1 of 3 tickets bought amongst 5 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))*(1-(1-p52))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))*(1-(1-p52))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))*(1-(1-p52))
term4 = p41*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)*(1-(1-p52))
term5 = p51*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-(1-p41))*(1-p51)

# 2 of 2 tickets bought amongst 5 persons
term6 = (p11+p22)*(1-p11)*(1-p22)
term7 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)
term8 = (p11+p42)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-p42)
term9 = (p11+p52)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))*(1-p52)
term10 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)
term11 = (p21+p42)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-p42)
term12 = (p21+p52)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))*(1-p52)
term13 = (p31+p42)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-p42)
term14 = (p31+p52)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))*(1-p52)
term15 = (p41+p52)*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)*(1-p52)



# Expectation
expectation = term1 + term2 + term3 + term4 + \
              term5 + term6 + term7 + term8 + \
              term9 + term10 + term11 + term12 + \
              term13 + term15

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("5persons2tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)
table = np.zeros(shape=(5,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.618075
p21: 0.551353
p31: 0.666734
p41: 0.502064
p51: 0.5
p22: 0.739551
p32: 0.692173
p42: 0.56214
p52: 0.564308

 == Corresponding to: == 
[[0.61808 0.     ]
 [0.55135 0.73955]
 [0.66673 0.69217]
 [0.50206 0.56214]
 [0.5     0.56431]]


# 3 tickets - 5 persons: (maybe wrong equations - double check)

In [410]:
p11, p21, p31, p41, p51 = sp.symbols('p11, p21, p31, p41, p51')
p22, p32, p42, p52      = sp.symbols('p22, p32, p42, p52')
p33, p43, p53           = sp.symbols('p33, p43, p53')

vars = [p11, p21, p31, p41, p51, p22, p32, p42, p52, p33, p43, p53]

# Only 1 of 3 tickets bought amongst 5 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))*(1-(1-p52))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))*(1-(1-p52))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))*(1-(1-p52))
term4 = p41*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)*(1-(1-p52))
term5 = p51*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-(1-p41))*(1-p51)

# 2 of 2 tickets bought amongst 5 persons
term6 = (p11+p22)*(1-p11)*(1-p22)*(1-(1-p33))*(1-(1-p43))*(1-(1-p53))
term7 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)*(1-(1-p43))*(1-(1-p53))
term8 = (p11+p42)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-p42)*(1-(1-p53))
term9 = (p11+p52)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))*(1-p52)
term10 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)*(1-(1-p43))*(1-(1-p53))
term11 = (p21+p42)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-p42)*(1-(1-p53))
term12 = (p21+p52)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))*(1-p52)
term13 = (p31+p42)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-p42)*(1-(1-p53))
term14 = (p31+p52)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))*(1-p52)
term15 = (p41+p52)*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)*(1-p52)

# 3 of 3 tickets bought amongst 5 persons
term16 = (p11+p22+p33)*(1-p11)*(1-p22)*(1-p33)
term17 = (p11+p22+p43)*(1-p11)*(1-p22)*(1-(1-p33))*(1-p43)
term18 = (p11+p22+p53)*(1-p11)*(1-p22)*(1-(1-p33))*(1-(1-p43))*(1-p53)
term19 = (p11+p32+p43)*(1-p11)*(1-(1-p22))*(1-p32)*(1-p43)
term20 = (p11+p32+p53)*(1-p11)*(1-(1-p22))*(1-p32)*(1-(1-p43))*(1-p53)
term21 = (p11+p42+p53)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-p42)*(1-p53)
term22 = (p21+p32+p43)*(1-(1-p11))*(1-p21)*(1-p32)*(1-p43)
term23 = (p21+p32+p53)*(1-(1-p11))*(1-p21)*(1-p32)*(1-(1-p43))*(1-p53)
term24 = (p21+p42+p53)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-p42)*(1-p53)
term25 = (p31+p42+p53)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-p42)*(1-p53)


# Expectation
expectation = term1 + term2 + term3 + term4 + \
              term5 + term6 + term7 + term8 + \
              term9 + term10 + term11 + term12 + \
              term13 + term15 + term16 + term17 + \
              term18 + term19 + term20 + term21 + \
              term22 + term23 + term24 + term25

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("5persons3tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)
table = np.zeros(shape=(5,3))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.53614
p21: 0.477261
p31: 0.568039
p41: 0.50214
p51: 0.5
p22: 0.605658
p32: 0.551507
p42: 0.433065
p52: 0.565909
p33: 0.695318
p43: 0.624999
p53: 0.5

 == Corresponding to: == 
[[0.53614 0.      0.     ]
 [0.47726 0.60566 0.     ]
 [0.56804 0.55151 0.69532]
 [0.50214 0.43307 0.625  ]
 [0.5     0.56591 0.5    ]]
